In [2]:
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import requests
import torch
import time

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
#from torchinfo import summary
import numpy as np
import os
import json

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Read lines from the text file and extract English-French sentence pairs
english_to_french  = [
    ("I am cold", "J'ai froid"),
    ("You are tired", "Tu es fatigué"),
    ("He is hungry", "Il a faim"),
    ("She is happy", "Elle est heureuse"),
    ("We are friends", "Nous sommes amis"),
    ("They are students", "Ils sont étudiants"),
    ("The cat is sleeping", "Le chat dort"),
    ("The sun is shining", "Le soleil brille"),
    ("We love music", "Nous aimons la musique"),
    ("She speaks French fluently", "Elle parle français couramment"),
    ("He enjoys reading books", "Il aime lire des livres"),
    ("They play soccer every weekend", "Ils jouent au football chaque week-end"),
    ("The movie starts at 7 PM", "Le film commence à 19 heures"),
    ("She wears a red dress", "Elle porte une robe rouge"),
    ("We cook dinner together", "Nous cuisinons le dîner ensemble"),
    ("He drives a blue car", "Il conduit une voiture bleue"),
    ("They visit museums often", "Ils visitent souvent des musées"),
    ("The restaurant serves delicious food", "Le restaurant sert une délicieuse cuisine"),
    ("She studies mathematics at university", "Elle étudie les mathématiques à l'université"),
    ("We watch movies on Fridays", "Nous regardons des films le vendredi"),
    ("He listens to music while jogging", "Il écoute de la musique en faisant du jogging"),
    ("They travel around the world", "Ils voyagent autour du monde"),
    ("The book is on the table", "Le livre est sur la table"),
    ("She dances gracefully", "Elle danse avec grâce"),
    ("We celebrate birthdays with cake", "Nous célébrons les anniversaires avec un gâteau"),
    ("He works hard every day", "Il travaille dur tous les jours"),
    ("They speak different languages", "Ils parlent différentes langues"),
    ("The flowers bloom in spring", "Les fleurs fleurissent au printemps"),
    ("She writes poetry in her free time", "Elle écrit de la poésie pendant son temps libre"),
    ("We learn something new every day", "Nous apprenons quelque chose de nouveau chaque jour"),
    ("The dog barks loudly", "Le chien aboie bruyamment"),
    ("He sings beautifully", "Il chante magnifiquement"),
    ("They swim in the pool", "Ils nagent dans la piscine"),
    ("The birds chirp in the morning", "Les oiseaux gazouillent le matin"),
    ("She teaches English at school", "Elle enseigne l'anglais à l'école"),
    ("We eat breakfast together", "Nous prenons le petit déjeuner ensemble"),
    ("He paints landscapes", "Il peint des paysages"),
    ("They laugh at the joke", "Ils rient de la blague"),
    ("The clock ticks loudly", "L'horloge tic-tac bruyamment"),
    ("She runs in the park", "Elle court dans le parc"),
    ("We travel by train", "Nous voyageons en train"),
    ("He writes a letter", "Il écrit une lettre"),
    ("They read books at the library", "Ils lisent des livres à la bibliothèque"),
    ("The baby cries", "Le bébé pleure"),
    ("She studies hard for exams", "Elle étudie dur pour les examens"),
    ("We plant flowers in the garden", "Nous plantons des fleurs dans le jardin"),
    ("He fixes the car", "Il répare la voiture"),
    ("They drink coffee in the morning", "Ils boivent du café le matin"),
    ("The sun sets in the evening", "Le soleil se couche le soir"),
    ("She dances at the party", "Elle danse à la fête"),
    ("We play music at the concert", "Nous jouons de la musique au concert"),
    ("He cooks dinner for his family", "Il cuisine le dîner pour sa famille"),
    ("They study French grammar", "Ils étudient la grammaire française"),
    ("The rain falls gently", "La pluie tombe doucement"),
    ("She sings a song", "Elle chante une chanson"),
    ("We watch a movie together", "Nous regardons un film ensemble"),
    ("He sleeps deeply", "Il dort profondément"),
    ("They travel to Paris", "Ils voyagent à Paris"),
    ("The children play in the park", "Les enfants jouent dans le parc"),
    ("She walks along the beach", "Elle se promène le long de la plage"),
    ("We talk on the phone", "Nous parlons au téléphone"),
    ("He waits for the bus", "Il attend le bus"),
    ("They visit the Eiffel Tower", "Ils visitent la tour Eiffel"),
    ("The stars twinkle at night", "Les étoiles scintillent la nuit"),
    ("She dreams of flying", "Elle rêve de voler"),
    ("We work in the office", "Nous travaillons au bureau"),
    ("He studies history", "Il étudie l'histoire"),
    ("They listen to the radio", "Ils écoutent la radio"),
    ("The wind blows gently", "Le vent souffle doucement"),
    ("She swims in the ocean", "Elle nage dans l'océan"),
    ("We dance at the wedding", "Nous dansons au mariage"),
    ("He climbs the mountain", "Il gravit la montagne"),
    ("They hike in the forest", "Ils font de la randonnée dans la forêt"),
    ("The cat meows loudly", "Le chat miaule bruyamment"),
    ("She paints a picture", "Elle peint un tableau"),
    ("We build a sandcastle", "Nous construisons un château de sable"),
    ("He sings in the choir", "Il chante dans le chœur"),
    ("They ride bicycles", "Ils font du vélo"),
    ("The coffee is hot", "Le café est chaud"),
    ("She wears glasses", "Elle porte des lunettes"),
    ("We visit our grandparents", "Nous rendons visite à nos grands-parents"),
    ("He plays the guitar", "Il joue de la guitare"),
    ("They go shopping", "Ils font du shopping"),
    ("The teacher explains the lesson", "Le professeur explique la leçon"),
    ("She takes the train to work", "Elle prend le train pour aller au travail"),
    ("We bake cookies", "Nous faisons des biscuits"),
    ("He washes his hands", "Il se lave les mains"),
    ("They enjoy the sunset", "Ils apprécient le coucher du soleil"),
    ("The river flows calmly", "La rivière coule calmement"),
    ("She feeds the cat", "Elle nourrit le chat"),
    ("We visit the museum", "Nous visitons le musée"),
    ("He fixes his bicycle", "Il répare son vélo"),
    ("They paint the walls", "Ils peignent les murs"),
    ("The baby sleeps peacefully", "Le bébé dort paisiblement"),
    ("She ties her shoelaces", "Elle attache ses lacets"),
    ("We climb the stairs", "Nous montons les escaliers"),
    ("He shaves in the morning", "Il se rase le matin"),
    ("They set the table", "Ils mettent la table"),
    ("The airplane takes off", "L'avion décolle"),
    ("She waters the plants", "Elle arrose les plantes"),
    ("We practice yoga", "Nous pratiquons le yoga"),
    ("He turns off the light", "Il éteint la lumière"),
    ("They play video games", "Ils jouent aux jeux vidéo"),
    ("The soup smells delicious", "La soupe sent délicieusement bon"),
    ("She locks the door", "Elle ferme la porte à clé"),
    ("We enjoy a picnic", "Nous profitons d'un pique-nique"),
    ("He checks his email", "Il vérifie ses emails"),
    ("They go to the gym", "Ils vont à la salle de sport"),
    ("The moon shines brightly", "La lune brille intensément"),
    ("She catches the bus", "Elle attrape le bus"),
    ("We greet our neighbors", "Nous saluons nos voisins"),
    ("He combs his hair", "Il se peigne les cheveux"),
    ("They wave goodbye", "Ils font un signe d'adieu")
]

def save_metrics(metrics, filename):
    """ Save metrics as a JSON file """
    with open(filename, 'w') as f:
        json.dump(metrics, f, indent=4)

def plot_metrics(metrics, filename, title, ylabel):
    """ Plot and save metrics as an image """
    plt.figure()
    plt.plot(metrics, linestyle='-', label='Metric')
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid()
    plt.savefig(filename)
    plt.close()

class Vocab:
    def __init__(self):
        # Initialize dictionaries for word to index and index to word mappings
        self.word2index = {"SOS": SOS_token, "EOS": EOS_token}
        self.index2word = {SOS_token:"SOS", EOS_token: "EOS"}
        self.word_count = {}  # Keep track of word frequencies
        self.n_words = 2  # Start counting from 3 to account for special tokens

    def add_sentence(self, sentence):
        # Add all words in a sentence to the vocabulary
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        # Add a word to the vocabulary
        if word not in self.word2index:
            # Assign a new index to the word and update mappings
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word
            self.word_count[word] = 1
            self.n_words += 1
        else:
            # Increment word count if the word already exists in the vocabulary
            self.word_count[word] += 1

# Custom Dataset class for English to French sentences
class TranslationDataset(Dataset):
    def __init__(self, pairs):
        self.first_vocab = Vocab()
        self.second_vocab = Vocab()
        self.pairs = []

        # Process each English-French pair
        for first, fr in pairs:
            self.first_vocab.add_sentence(first)
            self.second_vocab.add_sentence(fr)
            self.pairs.append((first, fr))

        # Separate English and French sentences
        self.first_sentences = [pair[0] for pair in self.pairs]
        self.second_sentences = [pair[1] for pair in self.pairs]

    def __len__(self):
        # Return the number of sentence pairs
        return len(self.pairs)

    def __getitem__(self, idx):
        # Get the sentences by index
        first_sentence = self.first_sentences[idx]
        second_sentence = self.second_sentences[idx]
        input_indices = [self.first_vocab.word2index[word] for word in first_sentence.split()] + [EOS_token]
        target_indices = [self.second_vocab.word2index[word] for word in second_sentence.split()] + [EOS_token]

        return torch.tensor(input_indices, dtype=torch.long), torch.tensor(target_indices, dtype=torch.long)

SOS_token = 0
EOS_token = 1

dataset = TranslationDataset(english_to_french)
train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
valid_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)


class Encoder(nn.Module):
    """The Encoder part of the seq2seq model."""
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        # Forward pass for the encoder
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def initHidden(self):
        # Initializes hidden state
        return torch.zeros(1, 1, self.hidden_size, device=device)

class Decoder(nn.Module):
    """The Decoder part of the seq2seq model."""
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

def training_func(encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, dataloader, epochs, device, results_dir):

    training_losses = []

    for epoch in range(epochs):
        total_loss = 0
        correct_train = 0

        for i, (input_tensor, target_tensor) in enumerate(dataloader):
            # Move tensors to the correct device
            input_tensor = input_tensor[0].to(device)
            target_tensor = target_tensor[0].to(device)

            # Initialize encoder hidden state
            encoder_hidden = encoder.initHidden()

            # Clear gradients for optimizers
            encoder_optimizer.zero_grad()
            decoder_optimizer.zero_grad()

            # Calculate the length of input and target tensors
            input_length = input_tensor.size(0)
            target_length = target_tensor.size(0)

            # Initialize loss
            loss = 0

            # Encoding each word in the input
            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)

            # Decoder's first input is the SOS token
            decoder_input = torch.tensor([[SOS_token]], device=device)

            # Decoder starts with the encoder's last hidden state
            decoder_hidden = encoder_hidden

            # Decoding loop
            for di in range(target_length):
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                # Choose top1 word from decoder's output
                topv, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze().detach()  # Detach from history as input

                # Calculate loss
                loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
                if decoder_input.item() == EOS_token:  # Stop if EOS token is generated
                    break

            # Backpropagation
            loss.backward()

            # Update encoder and decoder parameters
            encoder_optimizer.step()
            decoder_optimizer.step()

            #Average Loss
            total_loss += loss.item() / target_length

        epoch_loss = total_loss / len(dataloader)
        training_losses.append(total_loss / len(dataloader))

        # Print loss every 10 epochs
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {total_loss / len(dataloader)}')

    # Save training loss
    save_metrics(training_losses, os.path.join(results_dir, 'training_loss.json'))
    plot_metrics(training_losses, os.path.join(results_dir, 'training_loss.png'), 'Training Loss Over Epochs', 'Loss')



input_size = len(dataset.first_vocab.word2index)
hidden_size = 256
output_size =  len(dataset.second_vocab.word2index)

lr = 0.0001
epochs = 50

encoder = Encoder(input_size, hidden_size).to(device)
decoder = Decoder(hidden_size, output_size).to(device)

encoder_optimizer = optim.Adam(encoder.parameters(), lr=lr)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=lr)

criterion = nn.NLLLoss()

def validation_func(encoder, decoder, dataloader, criterion, results_dir, n_examples=10):
    # Switch model to evaluation mode
    encoder.eval()
    decoder.eval()

    second_vocab = dataloader.dataset.second_vocab
    first_vocab = dataloader.dataset.first_vocab

    total_loss = 0
    correct_predictions = 0
    evaluation_results = []

    # No gradient calculation
    with torch.no_grad():
        for i, (input_tensor, target_tensor) in enumerate(dataloader):
            # Move tensors to the correct device
            input_tensor = input_tensor[0].to(device)
            target_tensor = target_tensor[0].to(device)

            encoder_hidden = encoder.initHidden()

            input_length = input_tensor.size(0)
            target_length = target_tensor.size(0)

            loss = 0

            # Encoding step
            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)

            # Decoding step
            decoder_input = torch.tensor([[SOS_token]], device=device)
            decoder_hidden = encoder_hidden

            predicted_indices = []

            for di in range(target_length):
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                topv, topi = decoder_output.topk(1)
                predicted_indices.append(topi.item())
                decoder_input = topi.squeeze().detach()

                loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
                if decoder_input.item() == EOS_token:
                    break

            # Calculate and print loss and accuracy for the evaluation
            total_loss += loss.item() / target_length
            if predicted_indices == target_tensor.tolist():
                correct_predictions += 1

            if i < n_examples:
                evaluation_results.append({
                    'Input': input_tensor.tolist(),
                    'Target': target_tensor.tolist(),
                    'Predicted': predicted_indices
                })

            # Optionally, print some examples
            if i < n_examples:
                predicted_sentence = ' '.join([second_vocab.index2word[index] for index in predicted_indices if index not in (SOS_token, EOS_token)])
                target_sentence = ' '.join([second_vocab.index2word[index.item()] for index in target_tensor if index.item() not in (SOS_token, EOS_token)])
                input_sentence = ' '.join([first_vocab.index2word[index.item()] for index in input_tensor if index.item() not in (SOS_token, EOS_token)])

                print(f'Input: {input_sentence}, Target: {target_sentence}, Predicted: {predicted_sentence}')

        # Print overall evaluation results
        average_loss = total_loss / len(dataloader)
        accuracy = correct_predictions / len(dataloader)
        print(f'Evaluation Loss: {average_loss:.4f}, Accuracy: {100*accuracy:.2f}%')
        save_metrics({'Evaluation Loss': average_loss, 'Accuracy': accuracy}, os.path.join(results_dir, 'evaluation_results.json'))

# Create a directory to store results
results_dir = "Homework 4 Q1 Results"
os.makedirs(results_dir, exist_ok=True)

training_func(encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, train_dataloader, epochs, device, results_dir)
validation_func(encoder, decoder, valid_dataloader, criterion, results_dir)



Epoch 0, Loss: 4.189678477480734
Epoch 10, Loss: 2.378833041530694
Epoch 20, Loss: 1.917090537987125
Epoch 30, Loss: 1.2913504952995234
Epoch 40, Loss: 0.8197143668864849
Input: She waters the plants, Target: Elle arrose les plantes, Predicted: Elle arrose les plantes
Input: We talk on the phone, Target: Nous parlons au téléphone, Predicted: Nous parlons au téléphone
Input: He writes a letter, Target: Il écrit une lettre, Predicted: Il écrit une lettre
Input: We travel by train, Target: Nous voyageons en train, Predicted: Nous voyageons en train
Input: She writes poetry in her free time, Target: Elle écrit de la poésie pendant son temps libre, Predicted: Elle écrit de la poésie pendant pendant pendant pendant temps
Input: We enjoy a picnic, Target: Nous profitons d'un pique-nique, Predicted: Nous profitons d'un pique-nique
Input: The wind blows gently, Target: Le vent souffle doucement, Predicted: Le vent souffle doucement
Input: She swims in the ocean, Target: Elle nage dans l'océan, 